<a href="https://colab.research.google.com/github/nikitanaumenkoo/python-ab-testing-analysis/blob/main/A_B_testing_analysis_(portfolio_project_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Підключення до бази даних**


In [ ]:
!pip install --upgrade google-cloud-bigquery

from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.38.0
    Uninstalling google-cloud-bigquery-3.38.0:
      Successfully uninstalled google-cloud-bigquery-3.38.0


In [ ]:
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

### **2. Створення датасету**


In [ ]:
sql_query = """
with session_info as (


 select s.date,
        s.ga_session_id,
        sp.country,
        sp.device,
        sp.continent,
        sp.channel,
        ab.test,
        ab.test_group
  from `DA.ab_test` ab
  join `DA.session` s
  on ab.ga_session_id = s.ga_session_id
  join `DA.session_params` sp
  on sp.ga_session_id = ab.ga_session_id
),
session_with_orders as (
 select session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct o.ga_session_id) as session_with_orders
 from `DA.order` o
 join session_info
 on o.ga_session_id = session_info.ga_session_id
 group by session_info.date,
          session_info.country,
          session_info.device,
          session_info.continent,
          session_info.channel,
          session_info.test,
          session_info.test_group
),
events as (
 select session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        ep.event_name,
        count(ep.ga_session_id) as event_cnt
 from `DA.event_params` ep
 join session_info
 on ep.ga_session_id = session_info.ga_session_id
 group by session_info.date,
          session_info.country,
          session_info.device,
          session_info.continent,
          session_info.channel,
          session_info.test,
          session_info.test_group,
          ep.event_name
),
session as (
 select session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct session_info.ga_session_id) as session_cnt
 from session_info
 group by session_info.date,
          session_info.country,
          session_info.device,
          session_info.continent,
          session_info.channel,
          session_info.test,
          session_info.test_group
),
account as (
 select session_info.date,
        session_info.country,
        session_info.device,
        session_info.continent,
        session_info.channel,
        session_info.test,
        session_info.test_group,
        count(distinct acs.ga_session_id) as new_account_cnt
 from `DA.account_session` acs
 join session_info
 on acs.ga_session_id = session_info.ga_session_id
 group by session_info.date,
          session_info.country,
          session_info.device,
          session_info.continent,
          session_info.channel,
          session_info.test,
          session_info.test_group
)




select session_with_orders.date,
       session_with_orders.country,
       session_with_orders.device,
       session_with_orders.continent,
       session_with_orders.channel,
       session_with_orders.test,
       session_with_orders.test_group,
       'session_with_orders' as event_name,
       session_with_orders.session_with_orders as value
from session_with_orders
union all
select events.date,
       events.country,
       events.device,
       events.continent,
       events.channel,
       events.test,
       events.test_group,
       events.event_name,
       events.event_cnt as value
from events
union all
select session.date,
       session.country,
       session.device,
       session.continent,
       session.channel,
       session.test,
       session.test_group,
       'session' as event_name,
       session.session_cnt as value
from session
union all
select account.date,
       account.country,
       account.device,
       account.continent,
       account.channel,
       account.test,
       account.test_group,
       'new_account' as event_name,
       account.new_account_cnt as value
from account
"""

ab_test_df = client.query(sql_query).to_dataframe()

### **3. Огляд та обробка датасету**

In [ ]:
ab_test_df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,session_with_orders,1
1,2020-11-03,New Zealand,desktop,Oceania,Direct,2,2,session_with_orders,1
2,2020-11-04,Bulgaria,mobile,Europe,Paid Search,2,1,session_with_orders,1
3,2020-11-04,Kuwait,mobile,Asia,Organic Search,2,2,session_with_orders,1
4,2020-11-05,Serbia,desktop,Europe,Social Search,2,2,session_with_orders,1


In [ ]:
ab_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


Усі типи даних коректні, пропущених даних немає, датасет обробки не потребує.

### **4. Підготовка до розрахунків**

####1) Агрегація даних

In [ ]:
# 1. Список розрізів
dimensions = ['test', 'test_group']

# 2. Групуємо за тестом, групою та назвою події
agg_df = ab_test_df.groupby(dimensions + ['event_name'])['value'].sum().unstack()

# Прибираємо назву індексу колонок для чистоти
agg_df.columns.name = None

# 3. Перетворюємо індекси назад у колонки та заповнюємо пустоти нулями
agg_df = agg_df.reset_index().fillna(0)

# Дивимось результат
agg_df.head()

,test,test_group,add_payment_info,add_shipping_info,add_to_cart,begin_checkout,click,first_visit,new_account,page_view,...,select_item,select_promotion,session,session_start,session_with_orders,user_engagement,view_item,view_item_list,view_promotion,view_search_results
0,1,1,1988,3034,1395,3784,368,30596,3823,191543,...,543,1275,45362,45905,4514,171788,62335,27,29188,3678
1,1,2,2229,3221,1366,4021,353,30512,3681,198050,...,530,1323,45193,45649,4526,179081,65337,24,29117,3882
2,2,1,2344,3480,2811,4262,337,34511,4165,220275,...,905,1477,50637,51219,5102,198266,72717,24,32367,4282
3,2,2,2409,3510,3061,4313,413,34171,4184,212320,...,946,1406,50244,50808,5003,189931,68700,29,31680,4198
4,3,1,3623,5298,17674,9532,280,50438,5856,286351,...,8735,2020,70047,71312,6951,249921,93931,9,41169,5764


####2) Налаштування розрахунків

In [ ]:
# Визначаємо метрики
metrics_dict = {
    'add_payment_info_rate': ('add_payment_info', 'session'),
    'add_shipping_info_rate': ('add_shipping_info', 'session'),
    'begin_checkout_rate': ('begin_checkout', 'session'),
    'new_accounts_rate': ('new_account', 'session')
}

# Створюємо список івентів, які нам точно потрібні (чисельники + знаменник)
required_columns = set()
for num, den in metrics_dict.values():
    required_columns.add(num)
    required_columns.add(den)

### **5. Підготовка фінальної структури та розрахунок**

####1) Формування структури

In [ ]:
# 1. Визначаємо колонки, за якими ми сегментуємо (без групи тесту)
segment_columns = ['test']

# 2. Отримуємо список унікальних сегментів, які існують у даних
segments = agg_df[segment_columns].drop_duplicates()

# 3. Імпортуємо інструмент для стат-тесту
from statsmodels.stats.proportion import proportions_ztest

print(f"Знайдено {len(segments)} унікальних сегментів для аналізу.")

Знайдено 4 унікальних сегментів для аналізу.


####2) Розрахунок

In [ ]:
final_results = []

# Проходимо циклом тільки по унікальних номерах тестів
for _, segment in segments.iterrows():
    # Фільтруємо дані ТІЛЬКИ за номером тесту
    current_data = agg_df[agg_df['test'] == segment['test']]

    # Виділяємо групи А (1) та B (2)
    group_a = current_data[current_data['test_group'] == 1]
    group_b = current_data[current_data['test_group'] == 2]

    # Перевірка наявності обох груп для порівняння
    if group_a.empty or group_b.empty:
        continue

    # Розрахунок для кожної метрики зі словника metrics_dict
    for metric_name, (num_ev, den_ev) in metrics_dict.items():
        # Збираємо дані (чисельник та знаменник)
        n_count_a = group_a[num_ev].values[0]
        d_count_a = group_a[den_ev].values[0]

        n_count_b = group_b[num_ev].values[0]
        d_count_b = group_b[den_ev].values[0]

        # Розрахунки конверсії та апліфту
        if d_count_a > 0 and d_count_b > 0:
            cr_a = n_count_a / d_count_a
            cr_b = n_count_b / d_count_b
            uplift = (cr_b - cr_a) / cr_a * 100 if cr_a != 0 else 0

            # Статистичний Z-тест
            z_stat, p_val = proportions_ztest([n_count_a, n_count_b], [d_count_a, d_count_b])
        else:
            cr_a, cr_b, uplift, z_stat, p_val = 0, 0, 0, 0, np.nan

        # Зберігаємо результат
        final_results.append({
            'test_number': segment['test'],
            'metric': metric_name,
            'numerator_event': num_ev,
            'denominator_event': den_ev,
            'numerator_count_a': n_count_a,
            'denominator_count_a': d_count_a,
            'conversion_rate_a': cr_a,
            'numerator_count_b': n_count_b,
            'denominator_count_b': d_count_b,
            'conversion_rate_b': cr_b,
            'metric_change_pct': uplift,
            'z_stat': z_stat,
            'p_value': p_val,
            'significant': True if p_val < 0.05 else False
        })

# Створюємо фінальний DataFrame
results_df = pd.DataFrame(final_results)

In [ ]:
results_df.head()

,test_number,metric,numerator_event,denominator_event,numerator_count_a,denominator_count_a,conversion_rate_a,numerator_count_b,denominator_count_b,conversion_rate_b,metric_change_pct,z_stat,p_value,significant
0,1,add_payment_info_rate,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,1,add_shipping_info_rate,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
2,1,begin_checkout_rate,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
3,1,new_accounts_rate,new_account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
4,2,add_payment_info_rate,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False


In [ ]:
# Збереження в CSV
results_df.to_csv("bigquery_data_for_tableau.csv", index=False)

from google.colab import files
files.download("bigquery_data_for_tableau.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###**6. Опис етапів проекту та висновки.**

1. Підключення до бази даних: налаштування зв'язку з базою даних Google BigQuery для отримання даних.

2. Створення датасету: розробка SQL-запиту для формування датасету.

3. Огляд та обробка датасету: перевірка на пропуски, перевірка типів даних.

4. Підготовка до розрахунків: визначення ключових метрик (Conversion Rates) та групування даних.

5. Фінальна структура та розрахунок: застосування двостороннього Z-тесту для розрахунку p-values. Формування фінальної структури даних у CSV для візуалізації в Tableau.

####Висновки:
На основі аналізу 4-х тестів виявлено:  
1. Статистично значуще зростання для метрики **add_payment_info_rate** у тесті №1.   
2. Статистично значуще зростання для метрики **add_shipping_info_rate** у тесті №1.  
3. Статистично значуще зростання для метрики **begin_checkout_rate** у тесті №1 та татистично значуще зниження у тестах №3 та №4.  
4. Статистично значуще зниження для метрики **new_accounts_rate** у тесті №4.  

Інші тести не показали достатніх доказів для відхилення нульової гіпотези.

Посилання на фінальну таблицю https://drive.google.com/file/d/1YZESI1KLahG7n2-X6Ta4AC5ywI8bJuWT/view?usp=sharing

Посилання на дашборд https://public.tableau.com/app/profile/nikita.naumenko/viz/abtest_17609736217570/ABtest_1